In [1]:
from jax import jit, random
import pandas as pd
from datetime import datetime, timedelta
import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
from IPython.display import Markdown
from plotly.express.colors import qualitative as qual_colours
from pathlib import Path
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Markdown

from estival.sampling import tools as esamp

from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel
from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, plot_spaghetti
from emu_renewal.outputs import plot_uncertainty_patches, PANEL_SUBTITLES, plot_3d_spaghetti, plot_post_prior_comparison
from emu_renewal.calibration import StandardCalib
from emu_renewal.utils import get_adjust_idata_index, adjust_summary_cols

from plotting import plot_main
from utils import load_target_data, load_mobility_data, load_vaccination_data, load_variant_prevalence_data

In [2]:
# Set country
country = 'Malaysia'

In [3]:
# load data
target_data = load_target_data(country)
mobility_data = load_mobility_data(country)
vaccination_data = load_vaccination_data(country)
variant_data = load_variant_prevalence_data(country)

In [4]:
# Specify fixed parameters and get calibration data
proc_update_freq = 21
init_time = 50
data = target_data
pop = 33e6
analysis_start = datetime(2021, 5, 1)
analysis_end = datetime(2022, 4, 30)
init_start = analysis_start - timedelta(init_time)
init_end = analysis_start - timedelta(1)
select_data = data.loc[analysis_start: analysis_end]
init_data = data.resample("D").asfreq().interpolate().loc[init_start: init_end] / 7.0

In [ ]:
proc_fitter = CosineMultiCurve()
renew_model = RenewalModel(pop, analysis_start, analysis_end, proc_update_freq, proc_fitter, GammaDens(), init_time, init_data, GammaDens())

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.TruncatedNormal(7.3, 0.5, low=1.0),
    "gen_sd": dist.TruncatedNormal(3.8, 0.5, low=1.0),
    "cdr": dist.Beta(16, 40),
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.TruncatedNormal(8, 0.5, low=1.0),
    "report_sd": dist.TruncatedNormal(3, 0.5, low=1.0),
    "prop_immune": dist.Beta(3, 50)
}

In [ ]:
#| output: false
calib = StandardCalib(renew_model, priors, select_data, indicator='weekly_sum')
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=infer.init_to_uniform(radius=0.5))
mcmc = infer.MCMC(kernel, num_chains=4, num_samples=2000, num_warmup=200)
mcmc.run(random.PRNGKey(1))

In [ ]:
idata = az.from_dict(mcmc.get_samples(True))
idata_sampled = az.extract(idata, num_samples=800)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
# get model results and outputs
def get_full_result(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd, prop_immune):
    return renew_model.renewal_func(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd, prop_immune)

full_wrap = jit(get_full_result)
panel_subtitles = ["weekly_sum"] + PANEL_SUBTITLES[1:]
spaghetti = get_spaghetti_from_params(renew_model, sample_params, full_wrap, outputs=panel_subtitles)
quantiles_df = get_quant_df_from_spaghetti(renew_model, spaghetti, quantiles=[0.05, 0.5, 0.95], outputs=panel_subtitles)

In [ ]:
# Restrict range of mobility and vax data to analysis timeframe
mobility_data = mobility_data.loc[analysis_start:analysis_end]
vaccination_data = vaccination_data.loc[analysis_start:analysis_end]

In [ ]:
def plot(model_data, case_data):
    case_index = case_data.index
    cases = case_data
     
    # Define elements needed to add median line plots
    model_index = model_data.index
    weekly_median = model_data["weekly_sum"][0.50]
    rt_median = model_data["R"][0.50]
    suscept_median = model_data["susceptibles"][0.50]
    transmission_median = model_data["transmission potential"][0.50]
    
    
    # Define elements needed for uncertainty plots
    x_vals = model_data.index.to_list() + model_data.index[::-1].to_list()
    y_vals_weekly = model_data["weekly_sum"][0.05].to_list() + model_data["weekly_sum"][0.95][::-1].to_list()
    y_vals_R = model_data["R"][0.05].to_list() + model_data["R"][0.95][::-1].to_list()
    y_vals_suscept = model_data["susceptibles"][0.05].to_list() + model_data["susceptibles"][0.95][::-1].to_list()
    y_vals_transmission = model_data["transmission potential"][0.05].to_list() + model_data["transmission potential"][0.95][::-1].to_list()
                           
    # Create subplot
    fig = make_subplots(1, 2, shared_xaxes=True,  subplot_titles=(),
                       horizontal_spacing = 0.05, vertical_spacing = 0.05)
    
    # Add modelled case notifications median line
    fig.add_trace(go.Scatter(x=model_index, y=weekly_median, mode="lines", name="Modelled cases", marker_color='#636EFA', showlegend=False ), row=1, col=1)
    # Add modelled case notifications uncertainty
    fig.add_trace(go.Scatter(x=x_vals, y=y_vals_weekly, mode="lines", name="Modelled cases", line={"width": 0.0, "color": '#636EFA'}, fill='toself',
                             showlegend=False ), row=1, col=1)
    # Add case notifications
    fig.add_trace(go.Scatter(x=case_index, y=cases,  mode="markers", name="Reported cases", marker_color="black", showlegend=False ), row=1, col=1)
    
    # Add Rt median line
    fig.add_trace(go.Scatter(x=model_index, y=rt_median, mode="lines", name="Rt", marker_color='#00CC96', showlegend=False ), row=1, col=2)
    # Add Rt uncertainty 
    fig.add_trace(go.Scatter(x=x_vals, y=y_vals_R, mode="lines", name="Rt", line={"width": 0.0, "color": '#00CC96'}, fill='toself', showlegend=False ), row=1,
                  col=2)
                                 
    return fig.update_layout(height=500, width=1600) 

In [ ]:
fig = plot(quantiles_df, select_data)

number = 2

for i in range(number):
    fig.add_vrect(x0=datetime.strptime("2021-06-01", "%Y-%m-%d").timestamp() * 1000, x1=datetime.strptime("2021-06-16", "%Y-%m-%d").timestamp() * 1000,
             annotation_text="", annotation_position="right",
             fillcolor='#FFA15A', opacity=0.60, line_width=0.8, layer='below', row=1, col=i+1)
    
    delta = variant_data[variant_data['variant'] == 'Delta']
    delta_week = delta['week'].to_string(index=False)
    fig.add_vline(x=datetime.strptime(delta_week, "%Y-%m-%d").timestamp() * 1000, 
              annotation_text="Delta >50%", annotation_position="bottom right", annotation_font_size=20,
                  row=1, col=i+1, line_dash="dash")
    
    omicron = variant_data[variant_data['variant'] == 'Omicron_BA1_2']
    omicron_week = omicron['week'].to_string(index=False)
    fig.add_vline(x=datetime.strptime(omicron_week, "%Y-%m-%d").timestamp() * 1000, 
              annotation_text="Omicron >50%", annotation_position="bottom right", annotation_font_size=20,
                  row=1, col=i+1, line_dash="dash")

#fig['layout']['xaxis']['title']='Date'
#fig['layout']['xaxis1']['title']='Date'
fig['layout']['yaxis']['title']='Weekly reported cases'
fig['layout']['yaxis2']['title']='Reproduction number'

fig.update_layout(
    font=dict(
        size=18,  # Set the font size here
    )
)

fig.update_yaxes(
        title_standoff = 10)

fig.show()

In [ ]:
fig = plot_main(quantiles_df, select_data, mobility_data, vaccination_data).update_layout(showlegend=False)

# Add interventions and variant prevalence to plots
number = 2

for i in range(number):
    fig.add_vrect(x0=datetime.strptime("2021-06-01", "%Y-%m-%d").timestamp() * 1000, x1=datetime.strptime("2021-06-16", "%Y-%m-%d").timestamp() * 1000,
             annotation_text="", annotation_position="right",
             fillcolor='#FFA15A', opacity=0.60, line_width=0.8, layer='below', row=1, col=i+1)
    
    delta = variant_data[variant_data['variant'] == 'Delta']
    delta_week = delta['week'].to_string(index=False)
    fig.add_vline(x=datetime.strptime(delta_week, "%Y-%m-%d").timestamp() * 1000, 
              annotation_text="Delta >50%", annotation_position="bottom right", row=1, col=i+1, line_dash="dash")
    
    omicron = variant_data[variant_data['variant'] == 'Omicron_BA1_2']
    omicron_week = omicron['week'].to_string(index=False)
    fig.add_vline(x=datetime.strptime(omicron_week, "%Y-%m-%d").timestamp() * 1000, 
              annotation_text="Omicron >50%", annotation_position="bottom right", row=1, col=i+1, line_dash="dash")
    
# add lockdown to mobility figure too
fig.add_vrect(x0=datetime.strptime("2021-06-01", "%Y-%m-%d").timestamp() * 1000, x1=datetime.strptime("2021-06-16", "%Y-%m-%d").timestamp() * 1000,
             annotation_text="", annotation_position="right",
             fillcolor='#FFA15A', opacity=0.60, line_width=0.8, layer='below', row=3, col=1)

fig['layout']['xaxis5']['title']='Date'
fig['layout']['xaxis6']['title']='Date'
fig['layout']['yaxis']['title']='Weekly reported cases'
fig['layout']['yaxis2']['title']=''
fig['layout']['yaxis3']['title']='Total persons susceptible'
fig['layout']['yaxis4']['title']='' 
fig['layout']['yaxis5']['title']='% change from baseline'
fig['layout']['yaxis6']['title']='% total population' 


fig.show()

In [ ]:
# Quick code for exploring quantiles
print(quantiles_df.loc['2021-7-5':'2021-7-20'])

In [ ]:
# quick code for calculating final attack rate
suscepts = quantiles_df['susceptibles']
suscepts['AR_05'] = 1 - suscepts[0.05]/pop
suscepts['AR_5'] = 1 - suscepts[0.50]/pop
suscepts['AR_95'] = 1- suscepts[0.95]/pop

In [ ]:
suscepts.tail(5)

In [ ]:
Markdown('## Model description')

In [ ]:
Markdown(renew_model.get_description())

In [ ]:
Markdown('## Prior distributions')

### Generation distribution

In [ ]:
Markdown('## Calibration')

In [ ]:
Markdown(calib.get_description())

In [ ]:
summary = az.summary(idata)
#summary = adjust_summary_cols(summary)
#summary.index = summary.index.map(get_adjust_idata_index(renew_model))
#Markdown(summary.to_markdown())

In [ ]:
plot_post_prior_comparison(idata, [p for p in priors.keys() if p !="rt_init"],priors);

In [ ]:
az.summary(idata)

In [ ]:
param_sum = az.summary(idata)
params = [i for i in param_sum.index if not any(s in i for s in ["proc", "rt_init"])]
cols = [i for i in param_sum.columns if not any(s in i for s in ["mcse_"])]
short_param_sum = param_sum.loc[params, cols]
short_param_sum.columns = short_param_sum.columns.str.replace("_", " ").str.replace("ess", "ESS").str.replace("hdi", "HDI")
param_str_conv = {
    "cdr": "CDR",
    "gen_mean": "generation distribution mean (days)",
    "gen_sd": "generation distribution SD (days)",
    "report_mean": "reporting distribution mean (days)",
    "report_sd": "reporting distribution SD (days)",
}
col_str_conv = {
    "r hat": "R-hat",
    "sd": "SD",
}
short_param_sum = short_param_sum.rename(param_str_conv)
short_param_sum = short_param_sum.rename(col_str_conv, axis=1)
short_param_sum